# Video Afisha Events Bot
Ноутбук генерации видео для бота событий Калининграда.

**Версия из репозитория events-bot-new**

In [ ]:
NOTEBOOK_VERSION = 'v24.15-fix-mask'
print(f'🚀 Engine {NOTEBOOK_VERSION}')

# ==========================================
# 1. ИМПОРТЫ И НАСТРОЙКИ
# ==========================================
import warnings
warnings.filterwarnings("ignore")

# 1. УСТАНОВКА БИБЛИОТЕК
print("⏳ Installing libraries...", flush=True)
!pip install moviepy==1.0.3 numpy requests telethon > /dev/null

import os
import sys
import random
import time
import json
import math
import asyncio
import requests
import glob

# --- ВАЖНЫЕ ИМПОРТЫ ---
import numpy as np             
from PIL import Image, ImageFont, ImageDraw, ImageFilter

# Telethon & Secrets
from telethon import TelegramClient
from telethon.sessions import StringSession
from kaggle_secrets import UserSecretsClient

# MoviePy
os.environ["SDL_AUDIODRIVER"] = "dummy"
from moviepy.editor import *

# Функция логгинга
def log(msg):
    print(msg, flush=True)

log("🚀 Engine v24.15-fix-mask")

In [ ]:
# ==========================================
# 2. КОНФИГУРАЦИЯ И ЗАГРУЗКА ДАННЫХ
# ==========================================
# --- АВТО-ПОИСК ПАПКИ ДАТАСЕТА (FIX FOR BOT) ---
KAGGLE_INPUT_ROOT = "/kaggle/input"
# Ищем папки, начинающиеся с video-afisha-session-
session_dirs = sorted(glob.glob(os.path.join(KAGGLE_INPUT_ROOT, "video-afisha-session-*")), reverse=True)

if session_dirs:
    # Берем самую свежую папку сессии
    SOURCE_FOLDER = session_dirs[0]
    log(f"✅ DETECTED BOT SESSION: {SOURCE_FOLDER}")
elif os.path.exists(os.path.join(KAGGLE_INPUT_ROOT, "afisha-dataset-2")):
    # Фолбэк для локальных тестов
    SOURCE_FOLDER = os.path.join(KAGGLE_INPUT_ROOT, "afisha-dataset-2")
    log(f"⚠️ Using legacy dataset: {SOURCE_FOLDER}")
else:
    # Если совсем ничего нет, берем первую попавшуюся папку в input
    all_dirs = [d for d in glob.glob(os.path.join(KAGGLE_INPUT_ROOT, "*")) if os.path.isdir(d)]
    if all_dirs:
        SOURCE_FOLDER = all_dirs[0]
        log(f"⚠️ Using generic first folder: {SOURCE_FOLDER}")
    else:
        SOURCE_FOLDER = "/kaggle/input/afisha-dataset-2" # Чтобы не упало сразу, но упадет позже
        log("❌ CRITICAL: NO INPUT DATASET FOUND!")

WORKING_DIR = "/kaggle/working"

# Параметры видео
W, H = 1080, 1920
FPS = 30
AUDIO_START_SEC = 294   

SPLIT_RATIO = 0.6 
SPLIT_Y_COORD = int(H * SPLIT_RATIO) 

# Шрифты (ищем внутри найденной папки)
FONT_PATH = os.path.join(SOURCE_FOLDER, "BebasNeue-Bold.ttf")
if not os.path.exists(FONT_PATH):
    # Рекурсивный поиск, если структура папок отличается
    found = glob.glob(os.path.join(KAGGLE_INPUT_ROOT, "**/*.ttf"), recursive=True)
    FONT_PATH = found[0] if found else None
    if FONT_PATH: log(f"Generic font found: {FONT_PATH}")

# Цвета
COLOR_TITLE = 'white'
COLOR_ACCENT = '#f1c40f' 
COLOR_DETAILS = '#bdc3c7'
BG_STRIP_COLOR = (80, 20, 140, 255) 

# Дефолтный конфиг
DEFAULT_RAW_DATA = {
  "intro": {"count": 2, "text": "ТЕСТ ЗАГРУЗКИ"},
  "scenes": [
    {
      "about": "Тест: Файл из Telegram",
      "date": "25 декабря", "location": "Кэш Telegram",
      "images": ["https://files.catbox.moe/vg30ot.jpg"]
    }
  ]
}

In [ ]:
# --- ФУНКЦИИ ЗАГРУЗКИ ---

async def download_via_telegram(filenames_map):
    """Качает файлы из Избранного или Канала через Telethon"""
    if not filenames_map: return

    log(f"\n--- 🔵 Telegram: Ищем файлы ({len(filenames_map)} шт) ---")
    
    # 1. Читаем секреты
    try:
        secrets = UserSecretsClient()
        api_id = int(secrets.get_secret("TELEGRAM_API_ID"))
        api_hash = secrets.get_secret("TELEGRAM_API_HASH")
        session_str = secrets.get_secret("TELEGRAM_SESSION")
        
        channel_id = None
        try:
            cid = secrets.get_secret("SOURCE_CHANNEL_ID")
            if cid: channel_id = int(cid)
        except: pass
    except Exception as e:
        log(f"[SKIP] Нет секретов или ошибка сети: {e}")
        return

    # 2. Подключаемся
    try:
        client = TelegramClient(StringSession(session_str), api_id, api_hash)
        await client.connect()
        if not await client.is_user_authorized():
            log("[ERROR] ❌ Сессия невалидна! (Скрипт перейдет к HTTP)")
            await client.disconnect()
            return
        
        target = channel_id if channel_id else "me"
        target_name = "CHANNEL" if channel_id else "SAVED MESSAGES"
        log(f"   > Подключено. Ищем в: {target_name}")

        for fname, local_path in filenames_map.items():
            if os.path.exists(local_path): continue
            
            log(f"   > Поиск: {fname} ...")
            found_msg = None
            try:
                async for message in client.iter_messages(target, search=fname, limit=100):
                    if message.media:
                        found_msg = message
                        break
            except Exception as e:
                log(f"     [ERR] {e}")

            if found_msg:
                log(f"     [FOUND] Скачиваем...")
                await found_msg.download_media(file=local_path)
                log("     [DONE]")
            else:
                log("     [NOT FOUND]")
        
        await client.disconnect()

    except Exception as e:
        log(f"[TELEGRAM ERROR] {e}")

def download_via_http(url, local_path):
    """HTTP скачивание с User-Agent"""
    if os.path.exists(local_path): return True
    log(f"   > HTTP Fallback: {url} ...")
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/120.0.0.0 Safari/537.36'}
        r = requests.get(url, headers=headers, timeout=20)
        if r.status_code == 200:
            with open(local_path, 'wb') as f:
                f.write(r.content)
            log("     [OK]")
            return True
        else:
            log(f"     [FAIL] Status {r.status_code}")
            return False
    except Exception as e:
        log(f"     [ERR] {e}")
        return False

In [ ]:
# --- ORCHESTRATOR ---

async def main_loader():
    # 1. Поиск payload.json
    # Сначала ищем в определенной SOURCE_FOLDER
    payload_path = os.path.join(SOURCE_FOLDER, "payload.json")
    
    if not os.path.exists(payload_path):
         # Если нет, ищем глобально во всем input
         candidates = glob.glob(os.path.join(KAGGLE_INPUT_ROOT, "**/payload.json"), recursive=True)
         if candidates: payload_path = candidates[0]

    if payload_path and os.path.exists(payload_path):
        log(f"FOUND PAYLOAD: {payload_path}")
        with open(payload_path, 'r', encoding='utf-8') as f:
            raw = json.load(f)
    else:
        log("Using Internal Default Data")
        raw = DEFAULT_RAW_DATA

    tasks_telegram = {} 
    url_map = {}        
    
    scenes = raw.get("scenes", [])
    for idx, scene in enumerate(scenes):
        imgs = scene.get("images", [])
        if imgs and isinstance(imgs, list):
            url = imgs[0]
            if url.startswith("http"):
                fname = url.split("/")[-1] 
                local_name = f"scene_{idx}_{fname}"
                local_path = os.path.join(WORKING_DIR, local_name)
                
                tasks_telegram[fname] = local_path
                url_map[url] = local_path

    if tasks_telegram:
        await download_via_telegram(tasks_telegram)
    
    log("\n--- 🟠 Финализация (HTTP) ---")
    final_assets = {}
    for url, path in url_map.items():
        if os.path.exists(path):
            final_assets[url] = os.path.abspath(path)
        else:
            if download_via_http(url, path):
                final_assets[url] = os.path.abspath(path)

    processed = {"intro": {}, "scenes": []}
    
    ri = raw.get("intro", {})
    cnt = ri.get("count")
    processed["intro"]["count"] = str(cnt) if cnt not in [None, 0, ""] else ""
    processed["intro"]["text"] = ri.get("text", "")
    processed["intro"]["cities"] = ri.get("cities", [])
    processed["intro"]["date"] = ri.get("date", "")
    processed["intro"]["pattern"] = ri.get("pattern", "STICKER")
    
    for scene in scenes:
        new_s = {
            "title": scene.get("about", scene.get("title", "")),
            "date": scene.get("date", ""),
            "location": scene.get("location", ""),
            "images": []
        }
        if scene.get("images"):
            u = scene["images"][0]
            if u in final_assets:
                new_s["images"].append(final_assets[u])
            elif not u.startswith("http"):
                new_s["images"].append(u) 
        processed["scenes"].append(new_s)
        
    return processed

# ЗАПУСК
CURRENT_SCENARIO = await main_loader()
SCENES_DATA = CURRENT_SCENARIO.get("scenes", [])
INTRO_DATA = CURRENT_SCENARIO.get("intro", {"count": "", "text": ""})

log("\n✅ Data Ready. Starting Render Engine...")
random.seed(time.time())

In [ ]:
# ==========================================
# 3. МАТЕМАТИКА
# ==========================================
def ease_out_cubic(t): 
    return 1 - (1 - t)**3

def ease_in_cubic(t): 
    return t * t * t

def ease_in_expo_aggressive(t):
    return 0 if t == 0 else pow(2, 12 * t - 12)

def ease_out_quint(t):
    return 1 - pow(1 - t, 5)

def ease_in_out_quint(t):
    if t < 0.5:
        return 16 * t * t * t * t * t
    else:
        return 1 - pow(-2 * t + 2, 5) / 2
        
def ease_out_expo(t):
    if t == 1: return 1
    return 1 - pow(2, -10 * t)

In [ ]:
# ==========================================
# 4. ТЕКСТОВЫЙ ДВИЖОК
# ==========================================
def get_font_object(font_path, fontsize):
    try:
        return ImageFont.truetype(font_path, fontsize) if font_path else ImageFont.load_default()
    except:
        return ImageFont.load_default()



def rgba_image_to_clip(img):
    rgba = np.array(img.convert('RGBA'))
    rgb = rgba[:, :, :3]
    clip = ImageClip(rgb)
    if rgba.shape[2] == 4:
        alpha = rgba[:, :, 3].astype('float32') / 255.0
        mask = ImageClip(alpha, ismask=True)
        # MoviePy blit expects 2D mask frames; keep the mask 2D.
        clip = clip.set_mask(mask)
    return clip

def create_word_clip_fixed_height(word, font, color, fixed_height, baseline_offset):
    w_text = font.getlength(word)
    pad_x = 10
    w_canvas = int(w_text + pad_x * 2)
    img = Image.new('RGBA', (w_canvas, fixed_height), (0, 0, 0, 0))
    draw = ImageDraw.Draw(img)
    
    rect_top = 5
    rect_bottom = fixed_height - 5
    draw.rectangle([0, rect_top, w_canvas, rect_bottom], fill=(10, 10, 10, 240))
    draw.text((pad_x, baseline_offset), word, font=font, fill=color)
    return rgba_image_to_clip(img)

def generate_kinetic_text(text, font_path, fontsize, color, start_time, duration, start_y):
    font = get_font_object(font_path, fontsize)
    try:
        left, top, right, bottom = font.getbbox("HgЙj")
        ascent, descent = -top, bottom
        line_height = ascent + descent + 5 
        baseline_y = ascent + 2
        clip_height = int(line_height + 10)
    except:
        clip_height = int(fontsize * 1.2)
        baseline_y = int(fontsize * 0.2)
    
    raw_lines = text.split('\n') if text else ['']
    lines = []
    space_w = font.getlength(' ')
    max_w = W - 100
    
    for raw_line in raw_lines:
        if not raw_line.strip():
            lines.append([])
            continue
        words = [w for w in raw_line.split(' ') if w]
        current_line = []
        current_w = 0
        for word in words:
            word_w = font.getlength(word)
            if current_line and current_w + word_w > max_w:
                lines.append(current_line)
                current_line = [word]
                current_w = word_w + space_w
            else:
                current_line.append(word)
                current_w += word_w + space_w
        if current_line:
            lines.append(current_line)

    word_clips = []
    current_y_pos = start_y
    word_global_index = 0
    
    fly_in_dur = 0.7 
    fly_in_dist = 200
    fall_dur = 0.4
    
    for line in lines:
        current_x = 50 
        for word in line:
            clip = create_word_clip_fixed_height(word, font, color, clip_height, baseline_y)
            
            delay = word_global_index * 0.05 
            word_start = start_time + delay
            
            life_duration = duration - delay
            if life_duration < (fall_dur + 0.1): life_duration = fall_dur + 0.1
            
            final_x = current_x
            final_y = current_y_pos
            
            def pos_func(t, fx=final_x, fy=final_y, dur=life_duration):
                if t < fly_in_dur:
                    prog = t / fly_in_dur
                    eased = ease_out_cubic(prog)
                    curr_y = (fy + fly_in_dist) - (fly_in_dist * eased)
                    return (fx, int(curr_y))
                elif t > (dur - fall_dur):
                    fall_t = t - (dur - fall_dur)
                    prog = min(fall_t / fall_dur, 1.0)
                    eased = ease_in_cubic(prog)
                    curr_y = fy + (250 * eased)
                    return (fx, int(curr_y))
                else:
                    return (fx, int(fy))

            clip = (clip
                    .set_start(word_start)
                    .set_duration(life_duration)
                    .set_position(pos_func)
                    .crossfadein(0.2)
                    .crossfadeout(fall_dur)) 
            
            word_clips.append(clip)
            real_w = clip.w 
            current_x += real_w + 5 
            word_global_index += 1
            
        current_y_pos += clip_height + 2 
        
    return word_clips, current_y_pos


In [ ]:
# ==========================================
# 5. ДВИЖОК 3D ТЕКСТА И БЕНТО
# ==========================================

def aspect_fill_crop(img, target_w, target_h):
    src_w, src_h = img.size
    src_ratio = src_w / src_h
    dst_ratio = target_w / target_h
    if src_ratio > dst_ratio:
        resize_h = target_h
        resize_w = int(target_h * src_ratio)
    else:
        resize_w = target_w
        resize_h = int(target_w / src_ratio)
    img_resized = img.resize((resize_w, resize_h), Image.LANCZOS)
    left = (resize_w - target_w) // 2
    top = (resize_h - target_h) // 2
    return img_resized.crop((left, top, left + target_w, top + target_h))

def make_3d_transform(start_x, start_y, start_scale, max_scale, t_start=0, speed_mult=1.0):
    screen_cx = W / 2
    screen_cy = H / 2
    vec_x = start_x - screen_cx
    vec_y = start_y - screen_cy
    
    def transform(t):
        if t < t_start: return 1.0, (int(start_x), int(start_y))
        prog = (t - t_start) * speed_mult
        if prog > 1.5: prog = 1.5
        factor = ease_in_expo_aggressive(prog/1.5)
        curr_scale = 1.0 + (max_scale - 1.0) * factor
        curr_x = screen_cx + vec_x * curr_scale
        curr_y = screen_cy + vec_y * curr_scale
        return curr_scale, (int(curr_x), int(curr_y))
    return transform

def create_single_word_clip(word, font_path, font_size, bg_color, text_color, target_height=None):
    font = get_font_object(font_path, font_size)
    
    pad_x = 25
    
    if target_height:
        # Режим фиксированной высоты (Аутро)
        h = target_height
        canvas_h = h
        w_text = font.getlength(word)
        bbox = font.getbbox(word)
        h_text_real = bbox[3] - bbox[1]
        pad_top = (canvas_h - h_text_real) // 2 - bbox[1] 
        y_pos = pad_top + 10
        canvas_w = int(w_text + pad_x*2)
    else:
        # Режим авто-высоты (Интро) - UPD v24.0
        
        # 1. Эталон
        bbox_sample = font.getbbox("Hg") 
        sample_h = bbox_sample[3] - bbox_sample[1]
        
        # 2. Еще более сжатый паддинг (1.25)
        fixed_h = int(sample_h * 1.25) 
        canvas_h = fixed_h
        
        w_text = font.getlength(word)
        canvas_w = int(w_text + pad_x*2)
        
        # 3. Базовая линия по эталону
        y_pos = (fixed_h - sample_h) // 2 - bbox_sample[1]

    img = Image.new('RGBA', (canvas_w, canvas_h), (0,0,0,0))
    draw = ImageDraw.Draw(img)
    
    draw.rectangle([0, 0, canvas_w, canvas_h], fill=bg_color)
    draw.text((pad_x, y_pos), word, font=font, fill=text_color)
    
    return rgba_image_to_clip(img), canvas_w, canvas_h

def parse_intro_text_to_structure(text):
    words = text.split()
    if not words:
        return [[("EVENTS", 30.0)]]

    lines_struct = []
    current_chunk = []
    
    for i, w in enumerate(words):
        is_long = len(w) > 4
        
        if is_long:
            if current_chunk:
                line_data = []
                for cw in current_chunk:
                    z = 25.0 if cw.isdigit() else 20.0
                    line_data.append((cw, z))
                lines_struct.append(line_data)
                current_chunk = []
            lines_struct.append([(w, 35.0)]) 
        else:
            current_chunk.append(w)
            if len(current_chunk) >= 2:
                line_data = []
                for cw in current_chunk:
                    line_data.append((cw, 25.0))
                lines_struct.append(line_data)
                current_chunk = []

    if current_chunk:
        line_data = []
        for cw in current_chunk:
            line_data.append((cw, 25.0))
        lines_struct.append(line_data)
                
    return lines_struct


In [ ]:
def create_overlay_cover(bento_images, intro_data):
    DURATION = 2.5
    
    # DEBUG: Print intro_data to see what we receive
    print(f"[DEBUG] intro_data keys: {intro_data.keys()}")
    print(f"[DEBUG] cities: {intro_data.get('cities', 'NOT FOUND')}")
    
    # --- 1. BENTO GRID (unchanged) ---
    layout = [
        (0, 0, 1, 1), (1, 0, 1, 1), (2, 0, 1, 1),
        (0, 1, 2, 2),               (2, 1, 1, 1), (2, 2, 1, 1),
        (0, 3, 1, 1), (1, 3, 2, 2), (0, 4, 1, 1)
    ]
    needed = len(layout)
    
    if not bento_images:
        pool = ["placeholder"] * needed
    else:
        pool = bento_images * (needed // len(bento_images) + 2)
        random.shuffle(pool)
    
    COL_COUNT, PAD = 3, 15
    GRID_W = 1200
    UNIT = (GRID_W - (COL_COUNT + 1) * PAD) // COL_COUNT
    START_X, START_Y = (W - GRID_W)//2, (H - (UNIT*5 + PAD*6))//2
    
    grid_clips = []
    
    def make_bento_logic(px, py, bw, bh, delay, speed):
        cx, cy = W/2, H/2
        def trans(t):
            if t < delay: return 1.0, (px, py)
            dt = (t - delay) * speed
            prog = min(dt / 2.0, 1.0) 
            scale = 1.0 - ease_in_cubic(prog)
            if scale < 0.01: scale = 0.01 
            curr_cx = (px+bw/2) + (cx - (px+bw/2)) * ease_in_cubic(prog)
            curr_cy = (py+bh/2) + (cy - (py+bh/2)) * ease_in_cubic(prog)
            return scale, (int(curr_cx - bw*scale/2), int(curr_cy - bh*scale/2))
        return trans

    for i, (col, row, ws, hs) in enumerate(layout):
        bw, bh = ws*UNIT + (ws-1)*PAD, hs*UNIT + (hs-1)*PAD
        px, py = START_X + PAD + col*(UNIT+PAD), START_Y + PAD + row*(UNIT+PAD)
        
        if pool[i] != "placeholder":
            path = os.path.join(SOURCE_FOLDER, pool[i])
            if os.path.exists(path):
                img = Image.open(path)
                img = aspect_fill_crop(img, bw, bh).filter(ImageFilter.GaussianBlur(3))
                
                clip = rgba_image_to_clip(img).set_duration(DURATION).set_start(0)
                if clip.mask is None:
                    clip = clip.add_mask()
                
                delay = random.uniform(0.0, 0.6) 
                speed = random.uniform(1.2, 2.5) 
                
                trans_func = make_bento_logic(px, py, bw, bh, delay, speed)
                
                moving = (clip.resize(lambda t, f=trans_func: f(t)[0])
                              .set_position(lambda t, f=trans_func: f(t)[1])
                              .crossfadeout(1.0)) 
                
                grid_clips.append(moving)

    # --- 2. STICKER PATTERN INTRO (v4 - Fixed cities) ---
    intro_text = str(intro_data.get("text", "СОБЫТИЯ")).upper()
    count_str = str(intro_data.get("count", ""))
    
    # Handle cities - can be list or None
    cities_raw = intro_data.get("cities", None)
    if isinstance(cities_raw, list):
        cities_list = cities_raw
    elif isinstance(cities_raw, str) and cities_raw:
        cities_list = [c.strip() for c in cities_raw.split(',') if c.strip()]
    else:
        cities_list = []
    
    print(f"[DEBUG] Parsed cities_list: {cities_list}")
    
    # --- Parse intro text to lines ---
    PREPOSITIONS = {'НА', 'В', 'ДО', 'ДЛЯ', 'ИЗ', 'ПО', 'С', 'К', 'О', 'У', 'ОТ', 'ЗА'}
    
    def group_words_with_prepositions(words):
        result = []
        i = 0
        while i < len(words):
            word = words[i]
            if word in PREPOSITIONS and i + 1 < len(words):
                result.append(word + ' ' + words[i + 1])
                i += 2
            else:
                result.append(word)
                i += 1
        return result
    
    def split_into_lines(words, max_lines=3):
        if len(words) <= 2:
            return [' '.join(words)]
        elif len(words) <= 4:
            mid = len(words) // 2
            return [' '.join(words[:mid]), ' '.join(words[mid:])]
        else:
            third = len(words) // 3
            return [
                ' '.join(words[:third]),
                ' '.join(words[third:third*2]),
                ' '.join(words[third*2:])
            ]
    
    # Parse text
    text_lines = []
    if '\n' in intro_text:
        text_lines = [l.strip() for l in intro_text.split('\n') if l.strip()]
    else:
        if ':' in intro_text:
            parts = intro_text.split(':', 1)
            text_lines.append(parts[0].strip() + ':')
            remaining = parts[1].strip()
        else:
            remaining = intro_text
        
        if remaining:
            words = remaining.split()
            grouped = group_words_with_prepositions(words)
            text_lines.extend(split_into_lines(grouped))
    
    # Build line configs with rotations
    rotations = [4, -3, 4, 4]
    
    lines_config = []
    for i, line_text in enumerate(text_lines[:3]):
        rot = rotations[i % len(rotations)]
        if len(line_text) > 25:
            size = 85
        elif len(line_text) > 15:
            size = 95
        else:
            size = 110
        lines_config.append({'text': line_text, 'size': size, 'rot': rot, 'is_cities': False})
    
    # Add cities line if available
    if cities_list:
        cities_str = ', '.join(cities_list[:4])
        lines_config.append({
            'text': cities_str,
            'size': 50,
            'rot': 4,
            'is_cities': True
        })
        print(f"[DEBUG] Added cities line: {cities_str}")
    else:
        print("[DEBUG] No cities to add")
    
    print(f"[DEBUG] Final lines_config count: {len(lines_config)}")
    
    # Sticker strip styling
    BG_ACCENT = (241, 156, 28, 255)
    TEXT_BLACK = (0, 0, 0, 255)
    
    def create_sticker_strip(text, font_size, rotation_deg, is_cities=False):
        font = get_font_object(FONT_PATH, font_size)
        
        ref_bbox = font.getbbox('ЙрАБВду0123456789')
        full_height = ref_bbox[3] - ref_bbox[1]
        
        pad_x = 24
        pad_y = 6 if is_cities else 18
        
        text_bbox = font.getbbox(text)
        text_w = text_bbox[2] - text_bbox[0]
        content_w = int(text_w + pad_x * 2)
        content_h = int(full_height + pad_y * 2)
        
        img = Image.new('RGBA', (content_w, content_h), (0,0,0,0))
        draw = ImageDraw.Draw(img)
        
        bevel_ratio = 0.25
        b_len = content_h * bevel_ratio
        points = [
            (0, 0),
            (content_w - b_len, 0),
            (content_w, b_len),
            (content_w, content_h),
            (0, content_h)
        ]
        draw.polygon(points, fill=BG_ACCENT)
        
        ref_center = (ref_bbox[1] + ref_bbox[3]) / 2
        ty = (content_h / 2) - ref_center
        tx = (content_w - text_w) // 2
        draw.text((tx, ty), text, font=font, fill=TEXT_BLACK)
        
        if rotation_deg != 0:
            img = img.rotate(rotation_deg, expand=True, resample=Image.BICUBIC)
        
        return img, rotation_deg, content_h
    
    # Create all strips
    strip_data = []
    max_strip_width = 0
    total_height = 0
    # Dynamic spacing like preview: 15% of content height
    
    for cfg in lines_config:
        strip_img, rot, orig_h = create_sticker_strip(
            cfg['text'], cfg['size'], cfg['rot'], cfg['is_cities']
        )
        strip_data.append({
            'img': strip_img,
            'rotation': rot,
            'orig_h': orig_h
        })
        if strip_img.width > max_strip_width:
            max_strip_width = strip_img.width
        total_height += strip_img.height
    
    # Estimate spacing for total height calculation (15% of avg height)
    if strip_data:
        avg_h = sum(d["orig_h"] for d in strip_data) / len(strip_data)
        est_spacing = int(avg_h * 0.15) * (len(strip_data) - 1)
        total_height += est_spacing
    
    # Scale up if block < 80% screen width
    target_width = int(W * 0.80)
    scale_factor = 1.0
    if max_strip_width < target_width and max_strip_width > 0:
        scale_factor = target_width / max_strip_width
        scale_factor = min(scale_factor, 1.5)
    
    scaled_total_height = int(total_height * scale_factor)
    start_y = (H - scaled_total_height) // 2
    
    # Animation timing
    HOLD_TIME = 0.8
    FLY_DURATION = 1.0
    
    # Premium easing with overshoot
    def ease_in_back(t, overshoot=1.70158):
        return t * t * ((overshoot + 1) * t - overshoot)
    
    strip_clips = []
    curr_y = start_y
    
    for idx, data in enumerate(strip_data):
        strip_img = data['img']
        rotation_deg = data['rotation']
        
        if scale_factor != 1.0:
            new_w = int(strip_img.width * scale_factor)
            new_h = int(strip_img.height * scale_factor)
            strip_img = strip_img.resize((new_w, new_h), Image.LANCZOS)
        
        strip_w, strip_h = strip_img.size
        
        init_x = (W - strip_w) // 2
        init_y = curr_y
        
        rad = math.radians(rotation_deg)
        fly_distance = W + strip_w + 200
        
        direction = 1 if rotation_deg > 0 else -1
        fly_dx = direction * fly_distance * math.cos(rad)
        fly_dy = -direction * fly_distance * math.sin(rad)
        
        stagger_delay = idx * 0.06
        
        def make_fly_out_transform(ix, iy, dx, dy, hold, fly_dur, stagger):
            def transform(t):
                if t < hold + stagger:
                    return (ix, iy)
                fly_t = t - (hold + stagger)
                if fly_t >= fly_dur:
                    return (int(ix + dx), int(iy + dy))
                prog = fly_t / fly_dur
                eased = ease_in_back(prog, overshoot=2.5)
                return (int(ix + dx * eased), int(iy + dy * eased))
            return transform
        
        pos_func = make_fly_out_transform(init_x, init_y, fly_dx, fly_dy, HOLD_TIME, FLY_DURATION, stagger_delay)
        
        clip = rgba_image_to_clip(strip_img).set_duration(DURATION).set_start(0)
        clip = clip.set_position(pos_func)
        
        strip_clips.append(clip)
        
        # Proportional spacing like preview
        buffer_y = int(data["orig_h"] * 0.15 * scale_factor)
        curr_y += strip_h + buffer_y

    # Dimmer
    dimmer = ColorClip(size=(W, H), color=(0,0,0)).set_opacity(0.5).set_duration(DURATION).set_start(0).crossfadeout(0.6)

    return CompositeVideoClip(grid_clips + [dimmer] + strip_clips, size=(W,H))


In [ ]:
# ==========================================
# 6. ГЕНЕРАТОР СЦЕН & OUTRO
# ==========================================
def create_advanced_scene(image_path, text_data, start_delay=0.0):
    T_ENTRY = 0.45   
    T_HOLD = 1.2   
    T_MOVE = 0.75   
    T_INFO = 2.5    
    T_EXIT = 0.45    
    TOTAL_DURATION = start_delay + T_ENTRY + T_HOLD + T_MOVE + T_INFO + T_EXIT
    
    # Защита на случай, если файла нет (хотя мы проверяем до вызова)
    if not os.path.exists(image_path):
        print(f"Warning: Image not found {image_path}, utilizing ColorClip placeholder.")
        img = ColorClip(size=(W, H), color=(30,30,30)).set_duration(TOTAL_DURATION)
    else:
        # Load via PIL to force RGB(A) frames even for grayscale sources.
        img_pil = Image.open(image_path).convert('RGBA')
        img = rgba_image_to_clip(img_pil).resize(width=W)
    
    def transform_func(t):
        if t < start_delay: return 0.4, 'center'
        adj_t = t - start_delay
        if adj_t < T_ENTRY:
            return 0.4 + (0.5 * ease_out_cubic(adj_t/T_ENTRY)), 'center'
        elif adj_t < (T_ENTRY+T_HOLD):
            return 0.9 + (0.1 * ((adj_t-T_ENTRY)/T_HOLD)), 'center'
        elif adj_t < (T_ENTRY+T_HOLD+T_MOVE):
            prog = (adj_t - (T_ENTRY+T_HOLD))/T_MOVE
            sy, ey = H/2, SPLIT_Y_COORD/2
            return 1.0, int(sy + (ey-sy)*ease_in_out_quint(prog) - img.h/2)
        elif adj_t < (TOTAL_DURATION-start_delay-T_EXIT):
            return 1.0, int(SPLIT_Y_COORD/2 - 10*(adj_t-(T_ENTRY+T_HOLD+T_MOVE)) - img.h/2)
        else:
            prog = (adj_t - (TOTAL_DURATION-start_delay-T_EXIT))/T_EXIT
            sy = SPLIT_Y_COORD/2 - 10*T_INFO
            return 1.0, int(sy + (-img.h - sy)*ease_in_cubic(prog) - img.h/2)

    moving_img = (img.resize(lambda t: transform_func(t)[0])
                     .set_position(lambda t: ('center', transform_func(t)[1]))
                     .set_duration(TOTAL_DURATION))
    
    curtain_h = H - SPLIT_Y_COORD 
    curtain = ColorClip(size=(W, curtain_h), color=(10,10,10))
    move_start_time = start_delay + T_ENTRY + T_HOLD
    
    def curtain_pos(t):
        if t < move_start_time: return ('center', H)
        if t < move_start_time+T_MOVE:
            prog = (t-move_start_time)/T_MOVE
            return ('center', int(H + (SPLIT_Y_COORD-H)*ease_in_out_quint(prog)))
        return ('center', SPLIT_Y_COORD)
        
    curtain = curtain.set_start(0).set_duration(TOTAL_DURATION).set_position(curtain_pos)

    text_start = move_start_time + T_MOVE*0.2
    sy = SPLIT_Y_COORD + 30
    
    txts = []
    t1, ny = generate_kinetic_text(text_data['title'], FONT_PATH, 90, COLOR_TITLE, text_start, T_INFO+1, sy)
    txts.extend(t1)
    t2, ny = generate_kinetic_text(text_data['date'], FONT_PATH, 50, COLOR_ACCENT, text_start+0.2, T_INFO+1, ny+30)
    txts.extend(t2)
    t3, _ = generate_kinetic_text(text_data['location'], FONT_PATH, 45, COLOR_DETAILS, text_start+0.4, T_INFO+1, ny+15)
    txts.extend(t3)

    return CompositeVideoClip([ColorClip((W,H), (0,0,0), duration=TOTAL_DURATION), moving_img, curtain] + txts)


def make_slide_anim(sx, ex, fy, delay):
    def slide_func(t):
        if t < delay: return (sx, fy) 
        anim_time = t - delay
        anim_dur = 0.8 
        if anim_time >= anim_dur: return (ex, fy)
        prog = anim_time / anim_dur
        val = ease_out_expo(prog) 
        curr_x = sx + (ex - sx) * val
        return (int(curr_x), fy)
    return slide_func

def create_outro_slide_in():
    DURATION = 3.5 
    bg = ColorClip(size=(W, H), color=(0,0,0)).set_duration(DURATION)
    
    words_conf = [
        {"text": "ПОЛЮБИТЬ", "side": "left", "delay": 0.0}, 
        {"text": "КАЛИНИНГРАД", "side": "right", "delay": 0.4}, 
        {"text": "АНОНСЫ", "side": "left", "delay": 0.8}
    ]
    clips = []
    
    STRIP_H = 210 
    GAP = 20
    STEP_Y = STRIP_H + GAP
    
    total_block_h = len(words_conf) * STEP_Y - GAP 
    start_y_block = (H - total_block_h) / 2
    
    for i, item in enumerate(words_conf):
        txt = item['text']
        delay = item['delay']
        side = item['side']
        
        clp, cw, ch = create_single_word_clip(txt, FONT_PATH, 160, BG_STRIP_COLOR, COLOR_TITLE, target_height=STRIP_H)
        
        final_x = (W - cw) // 2
        final_y = int(start_y_block + i * STEP_Y)
        
        start_x_pos = -cw - 100 if side == "left" else W + 100
        
        pos_func = make_slide_anim(start_x_pos, final_x, final_y, delay)
            
        final = (clp.set_duration(DURATION)
                    .set_start(0)
                    .set_position(pos_func))
        
        clips.append(final)
        
    return CompositeVideoClip([bg] + clips)

In [ ]:
# ==========================================
# 7. СБОРКА
# ==========================================
print("--- Rendering Full Promo v24.0 ---")

try:
    # 1. Сбор всех картинок для Бенто
    all_bento_images = []
    for scene in SCENES_DATA:
        images = scene.get('images', [])
        for img_name in images:
            if img_name:
                all_bento_images.append(img_name)
    
    # UPD v24.0: Сборка сцен на основе JSON массива
    main_scenes = []
    DELAY_SCENE_1 = 1.3
    
    if not SCENES_DATA:
        print("No scenes in scenario!")
    else:
        for i, scene_data in enumerate(SCENES_DATA):
            print(f"Scene {i+1}: {scene_data['title']}...")
            
            # Извлекаем картинки
            images = scene_data.get('images', [])
            
            if not images:
                print(f"  > Warning: No images defined for scene {i+1}. Skipping.")
                continue
                
            # Берем первую картинку для визуализации сцены (как просили)
            main_image_name = images[0]
            path = os.path.join(SOURCE_FOLDER, main_image_name)
            
            delay = DELAY_SCENE_1 if i == 0 else 0.0
            main_scenes.append(create_advanced_scene(path, scene_data, delay))

    if main_scenes:
        print("Outro (Slide-In)...")
        main_scenes.append(create_outro_slide_in())
        
        main_video = concatenate_videoclips(main_scenes, method="compose", padding=-0.5)
        
        print(f"Intro Overlay: {INTRO_DATA['text']} ({INTRO_DATA['count']})")
        # Передаем список картинок для бенто
        cover_overlay = create_overlay_cover(all_bento_images, INTRO_DATA)
        
        final_video = CompositeVideoClip([main_video, cover_overlay])

        audio_files = [f for f in os.listdir(SOURCE_FOLDER) if f.lower().endswith('.mp3')]
        if audio_files:
            audioclip = AudioFileClip(os.path.join(SOURCE_FOLDER, audio_files[0]))
            if audioclip.duration > AUDIO_START_SEC: 
                audioclip = audioclip.subclip(AUDIO_START_SEC)
            audioclip = audioclip.volumex(0.45) 
            final_video = final_video.set_audio(audioclip.set_duration(final_video.duration).audio_fadeout(1.0))

        OUTPUT_FILE = "promo_v24_final.mp4"
        print(f"Saving to: {OUTPUT_FILE}")
        
        final_video.write_videofile(
            OUTPUT_FILE, 
            fps=FPS, 
            codec='libx265', 
            preset='medium',
            ffmpeg_params=["-crf", "26", "-pix_fmt", "yuv420p", "-tag:v", "hvc1"],
            logger='bar'
        )
        print("\nDONE!")
    else:
        print("Nothing to render.")

except Exception as e:
    print(f"\nERROR: {e}")
    import traceback
    traceback.print_exc()